## Content-Based recommendation system

In [5]:
# import mouldes.
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

In [6]:
# Read Data.
movies_data = pd.read_csv(r"../Data/movies.csv")
reating_data = pd.read_csv(r"../Data/ratings.csv")

In [7]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_data['year'] = movies_data.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_data['year'] = movies_data.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_data['title'] = movies_data.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_data['title'] = movies_data['title'].apply(lambda x: x.strip())
# Spilit genres.
movies_data['genres'] = movies_data.genres.str.split('|')
movies_data

,Unnamed: 0,movieId,title,genres,year
0,0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995
2,2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995
3,3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995
4,4,5,Father of the Bride Part II (1995),[Comedy],1995
...,...,...,...,...,...
95,95,97,"Hate (Haine, La) (1995)","[Crime, Drama]",1995
96,96,98,Shopping (1994),"[Action, Thriller]",1994
97,97,99,Heidi Fleiss: Hollywood Madam (1995),[Documentary],1995
98,98,100,City Hall (1996),"[Drama, Thriller]",1996


In [8]:
# Create a copy of our Data.
ganre_movies_data = movies_data.copy()
# Creating a dataframe that includes all genres of movies.
for index, row in movies_data.iterrows():
    for g in row["genres"] :  
        ganre_movies_data.at[index,g] = 1
# converting None (nan) Data to zero.
ganre_movies_data = ganre_movies_data.fillna(0)
# Deleting some columns.
ganre_movies_data = ganre_movies_data.drop(["Unnamed: 0",'title','genres','year'],axis=1)

In [9]:
# Get rating value for each film that your user has rate to it.
user_input = reating_data[['userId','movieId','rating']][reating_data['userId']== 1][:6]
# Get all of ganres of  moives that your user has rate to it.
user_ganre = pd.DataFrame(columns=list(ganre_movies_data.columns))
for movi_id in user_input['movieId'] :
    user_ganre = pd.concat( [user_ganre, ganre_movies_data[list(ganre_movies_data.columns)][ganre_movies_data['movieId'] == movi_id ]])
# Add columns to Data frame.
user_ganre.columns = [list(ganre_movies_data.columns)]
# ADD "rating" to Data frame
user_ganre = user_ganre.join(user_input["rating"])
# Convert None (nan) Data to zero.
user_ganre = user_ganre.fillna(0)
user_ganre = user_ganre.reset_index(drop=True)
user_ganre

,"(movieId,)","(Adventure,)","(Animation,)","(Children,)","(Comedy,)","(Fantasy,)","(Romance,)","(Drama,)","(Action,)","(Crime,)","(Thriller,)","(Horror,)","(Mystery,)","(Sci-Fi,)","(IMAX,)","(Documentary,)","(War,)","(Musical,)",rating
0,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1,3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,70,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Get ganre value or each ganre.
ganre_values = user_ganre[[('Adventure',),   ('Animation',),    ('Children',),
            ('Comedy',),     ('Fantasy',),     ('Romance',),       ('Drama',),
            ('Action',),       ('Crime',),    ('Thriller',),      ('Horror',),
           ('Mystery',),      ('Sci-Fi',),        ('IMAX',), ('Documentary',),
               ('War',),     ('Musical',)]].values
# Get rating or each ganre for your user.
rating_values = user_ganre[['rating']].values
profile = ganre_values.transpose().dot(rating_values)
profile

array([[4.],
       [4.],
       [4.],
       [8.],
       [4.],
       [4.],
       [0.],
       [3.],
       [3.],
       [3.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [11]:
# Get value of every value for each film.
all_of_movies = ganre_movies_data[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
        'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery',
        'Sci-Fi', 'IMAX', 'Documentary', 'War', 'Musical']].values
# Create your recomandation matrice.
recomand = ((all_of_movies * profile[:,0]).sum(axis=1))/(profile[:,0].sum())
recomand


array([0.64864865, 0.32432432, 0.32432432, 0.32432432, 0.21621622,
       0.24324324, 0.32432432, 0.21621622, 0.08108108, 0.27027027,
       0.32432432, 0.21621622, 0.32432432, 0.        , 0.2972973 ,
       0.08108108, 0.10810811, 0.21621622, 0.21621622, 0.45945946,
       0.37837838, 0.16216216, 0.24324324, 0.        , 0.10810811,
       0.        , 0.10810811, 0.10810811, 0.21621622, 0.08108108,
       0.        , 0.08108108, 0.21621622, 0.10810811, 0.10810811,
       0.08108108, 0.        , 0.32432432, 0.32432432, 0.        ,
       0.        , 0.16216216, 0.        , 0.2972973 , 0.2972973 ,
       0.10810811, 0.08108108, 0.32432432, 0.10810811, 0.16216216,
       0.16216216, 0.32432432, 0.10810811, 0.32432432, 0.        ,
       0.54054054, 0.        , 0.32432432, 0.        , 0.32432432,
       0.08108108, 0.        , 0.2972973 , 0.32432432, 0.21621622,
       0.16216216, 0.        , 0.32432432, 0.21621622, 0.37837838,
       0.08108108, 0.21621622, 0.        , 0.10810811, 0.21621